In [46]:
import pandas as pd
#method_raw_text = pd.read_excel('/Users/angieryu2202/Desktop/IF_App_Study/Yes_Responses.xlsx')
method_raw_text = pd.read_excel('/Users/angieryu2202/Desktop/IF_App_Study/No_Responses.xlsx')

# Knowledge Related Sentences in Reviews - Co-Word Network

In [47]:
# replace all newlines from dataframe
method_raw_text = method_raw_text.replace('\n','', regex=True)

In [48]:
method_raw_text = method_raw_text.dropna()

In [49]:
import re
for line in method_raw_text['Responses']:
    substring = re.sub(r'[^\w\s]','',str(line))
    #substring = ''.join([i for i in str(substring) if not i.isdigit()])
    substring = str(substring).lower().replace("$", "").replace("/","")
    method_raw_text['Responses'] = method_raw_text['Responses'].replace(line, substring)  

In [50]:
def text_preprocessor (text_column, column_name):
    import re, string
    from nltk import word_tokenize, pos_tag
    from nltk.corpus import stopwords
    from stop_words import get_stop_words
    from nltk.stem import WordNetLemmatizer
    stop_words = list(get_stop_words('en'))         #About 900 stopwords
    nltk_words = list(stopwords.words('english')) #About 150 stopwords
    stop_words.extend(nltk_words)
    #remove_words = ['payment', 'price', 'debit', 'card', 'account', 'tbh', 'subscription', 'app', 'application', 'youtube', 'username', 'password', 'yr', 'ur', 'id', 'isnt', 'wouldnt', 'doesnt', 'accountyou', 'im', 'thats', 'logins', 'wont', 'didnt', 'ive', 'ill', 'youre']
    #stop_words.extend(remove_words)
    lemmatizer = WordNetLemmatizer() 
    globals()[column_name+'_filtered_sent']=[]
    for line in text_column:
        word_tokens = word_tokenize(re.sub('[%s]' % re.escape(string.punctuation), '', str(line)))
        nouns = [token for token, pos in pos_tag(word_tokens) if pos.startswith('NN')]
        adjectives = [token for token, pos in pos_tag(word_tokens) if pos.startswith('JJ')]
        lemmatized = []
        for noun in nouns:
            lemmatized.append(lemmatizer.lemmatize(noun))
        for adjective in adjectives:
            lemmatized.append(lemmatizer.lemmatize(adjective))
        filtered_sentence = [w for w in lemmatized if not w in stop_words]
        globals()[column_name+'_filtered_sent'].append(filtered_sentence)

In [51]:
text_preprocessor(method_raw_text['Responses'], 'reviews')

In [52]:
reviews_filtered_sent

[['info', 'blurb', 'article', 'short', 'full'],
 ['order', 'convenience', 'design'],
 ['number',
  'diet',
  'article',
  'number',
  'faq',
  'page',
  'intermittent',
  'fasting',
  'health',
  'concern',
  'number',
  'information',
  'ill',
  'number',
  'others',
  'information',
  'useful',
  'useful',
  'new',
  'many',
  'general',
  'le',
  'general'],
 ['article',
  'subject',
  'lot',
  'time',
  'assumption',
  'information',
  'question',
  'answer',
  'access',
  'research',
  'related',
  'irrelevant',
  'dont',
  'good',
  'authoritative',
  'short',
  'definitive',
  'quick',
  'easy',
  'supplemental'],
 ['graphic',
  'image',
  'text',
  'format',
  'time',
  'user',
  'idea',
  'kind',
  'information',
  'service',
  'app',
  'offer',
  'uiux',
  'intuitive',
  'full',
  'text',
  'much'],
 ['option',
  'home',
  'page',
  'article',
  'contains',
  'image',
  'amount',
  'text',
  'slide',
  'legibility',
  'information',
  'content',
  'method',
  'option',
  'opt

In [53]:
import scipy.sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
class Cooccurrence(CountVectorizer):
    """Co-ocurrence matrix
    Convert collection of raw documents to word-word co-ocurrence matrix

    Parameters
    ----------
    encoding : string, 'utf-8' by default.
        If bytes or files are given to analyze, this encoding is used to
        decode.

    ngram_range : tuple (min_n, max_n)
        The lower and upper boundary of the range of n-values for different
        n-grams to be extracted. All values of n such that min_n <= n <= max_n
        will be used.

    max_df: float in range [0, 1] or int, default=1.0

    min_df: float in range [0, 1] or int, default=1

    Example
    -------

    >> import Cooccurrence
    >> docs = ['this book is good',
               'this cat is good',
               'cat is good shit']
    >> model = Cooccurrence()
    >> Xc = model.fit_transform(docs)

    Check vocabulary by printing
    >> model.vocabulary_

    """

    def __init__(self, encoding='utf-8', ngram_range=(1, 1),
                 max_df=1.0, min_df=1, max_features=None,
                 stop_words=None, normalize=True, vocabulary=None):

        super(Cooccurrence, self).__init__(
            ngram_range=ngram_range,
            max_df=max_df,
            min_df=min_df,
            max_features=max_features,
            stop_words=stop_words,
            vocabulary=vocabulary
        )

        self.X = None

        self.normalize = normalize

    def fit_transform(self, raw_documents, y=None):
        """Fit cooccurrence matrix

        Parameters
        ----------
        raw_documents : iterable
            an iterable which yields either str, unicode or file objects

        Returns
        -------
        Xc : Cooccurrence matrix

        """
        X = super(Cooccurrence, self).fit_transform(raw_documents)
        self.X = X

        n_samples, n_features = X.shape

        Xc = (X.T * X)
        if self.normalize:
            g = sp.diags(1./Xc.diagonal())
            Xc = g * Xc
        else:
            Xc.setdiag(0)

        return Xc

    def vocab(self):
        tuples = super(Cooccurrence, self).get_feature_names()
        vocabulary=[]
        for e_tuple in tuples:
            tokens = e_tuple.split()
            for t in tokens:
                if t not in vocabulary:
                    vocabulary.append(t)

        return vocabulary

    def word_histgram(self):
        word_list = super(Cooccurrence, self).get_feature_names()
        count_list = self.X.toarray().sum(axis=0)
        return dict(zip(word_list,count_list))

In [54]:
from collections import Counter
from nltk import bigrams
from collections import defaultdict
import operator
import numpy as np

class BaseCooccurrence:
    INPUT=[list,str]
    OUTPUT=[list,tuple]

class CooccurrenceWorker(BaseCooccurrence):
    def __init__(self):
        name = 'cooccurrence'
        self.inst = Cooccurrence(ngram_range=(2, 2), stop_words='english')

    def __call__(self, *args, **kwargs):

        # bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), vocabulary={'awesome unicorns': 0, 'batman forever': 1})
        co_occurrences = self.inst.fit_transform(args[0])
        # print('Printing sparse matrix:', co_occurrences)
        # print(co_occurrences.todense())
        sum_occ = np.sum(co_occurrences.todense(), axis=0)
        # print('Sum of word-word occurrences:', sum_occ)

        # Converting itertor to set
        result = zip(self.inst.get_feature_names(), np.array(sum_occ)[0].tolist())
        result_set = list(result)
        return result_set, self.inst.vocab()

class CooccurrenceManager:
    def computeCooccurence(self, list):
        com = defaultdict(lambda: defaultdict(int))
        count_all = Counter()
        count_all1 = Counter()

        uniqueList = []
        for _array in list:
            for line in _array:
                for word in line:
                    if word not in uniqueList:
                        uniqueList.append(word)

                terms_bigram = bigrams(line)
                # Update the counter
                count_all.update(line)
                count_all1.update(terms_bigram)

                # Build co-occurrence matrix
                for i in range(len(line) - 1):
                    for j in range(i + 1, len(line)):
                        w1, w2 = sorted([line[i], line[j]])
                        if w1 != w2:
                            com[w1][w2] += 1

        com_max = []
        # For each term, look for the most common co-occurrent terms
        for t1 in com:
            t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
            for t2, t2_count in t1_max_terms:
                com_max.append(((t1, t2), t2_count))
        # Get the most frequent co-occurrences
        terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)

        return terms_max, uniqueList



In [55]:
co = CooccurrenceWorker()

In [56]:
documents = []
for sublist in reviews_filtered_sent:
    document = ",".join(sublist)
    documents.append(document)

In [57]:
#import itertools
#merged = list(itertools.chain(*mecab_nouns))
co_result, vocab = co.__call__(documents)

In [58]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.font_manager as fm
import platform
from matplotlib.ft2font import FT2Font
import matplotlib as mpl

class GraphMLCreator:

    def __init__(self):
        self.G = nx.Graph()

        # Hack: offset the most central node to avoid too much overlap
        self.rad0 = 0.3

    def createGraphML(self, co_occurrence, word_hist, vocabulary, file):
        G = nx.Graph()

        for obj in vocabulary:
            G.add_node(obj)
        # convert list to a single dictionary

        for pair in co_occurrence:
            node1 = ''
            node2 = ''
            for inner_pair in pair:

                if type(inner_pair) is tuple:
                    node1 = inner_pair[0]
                    node2 = inner_pair[1]
                elif type(inner_pair) is str:
                    inner_pair=inner_pair.split()
                    node1 = inner_pair[0]
                    node2 = inner_pair[1]
                elif type(inner_pair) is int:
                    #print ("X " + node1 + " == " + node2 + " == " + str(inner_pair) + " : " + str(tuple[node1]))
                    G.add_edge(node1, node2, weight=float(inner_pair))
                elif type(inner_pair) is float:
                    #print ("X " + node1 + " == " + node2 + " == " + str(inner_pair) + " : ")
                    G.add_edge(node1, node2, weight=float(inner_pair))
        for word in word_hist:
            G.add_node(word, count=word_hist[word])
        self.G = G
        print(self.G.number_of_nodes())
        nx.write_graphml(G, file)

    def createGraphMLWithThreshold(self, co_occurrence, word_hist, vocab, file, threshold=10.0):
        G = nx.Graph()

        filtered_word_list=[]
        for pair in co_occurrence:
            node1 = ''
            node2 = ''
            for inner_pair in pair:
                if type(inner_pair) is tuple:
                    node1 = inner_pair[0]
                    node2 = inner_pair[1]
                elif type(inner_pair) is str:
                    inner_pair=inner_pair.split()
                    node1 = inner_pair[0]
                    node2 = inner_pair[1]
                elif type(inner_pair) is int:
                    if float(inner_pair) >= threshold:
                        #print ("X " + node1 + " == " + node2 + " == " + str(inner_pair) + " : " + str(tuple[node1]))
                        G.add_edge(node1, node2, weight=float(inner_pair))
                        if node1 not in filtered_word_list:
                            filtered_word_list.append(node1)
                        if node2 not in filtered_word_list:
                            filtered_word_list.append(node2)
                elif type(inner_pair) is float:
                    if float(inner_pair) >= threshold:
                        #print ("X " + node1 + " == " + node2 + " == " + str(inner_pair) + " : ")
                        G.add_edge(node1, node2, weight=float(inner_pair))
                        if node1 not in filtered_word_list:
                            filtered_word_list.append(node1)
                        if node2 not in filtered_word_list:
                            filtered_word_list.append(node2)
        for word in word_hist:
            if word in filtered_word_list:
                G.add_node(word, count=word_hist[word])

        self.G = G
        print(self.G.number_of_nodes())
        nx.write_graphml(G, file)

    def centrality_layout(self):
        centrality = nx.eigenvector_centrality_numpy(self.G)
        """Compute a layout based on centrality.
        """
        # Create a list of centralities, sorted by centrality value
        cent = sorted(centrality.items(), key=lambda x:float(x[1]), reverse=True)
        nodes = [c[0] for c in cent]
        cent  = np.array([float(c[1]) for c in cent])
        rad = (cent - cent[0])/(cent[-1]-cent[0])
        rad = self.rescale_arr(rad, self.rad0, 1)
        angles = np.linspace(0, 2*np.pi, len(centrality))
        layout = {}
        for n, node in enumerate(nodes):
            r = rad[n]
            th = angles[n]
            layout[node] = r*np.cos(th), r*np.sin(th)
        return layout

    def plot_graph(self, title=None, file='graph.png'):
        from matplotlib.font_manager import _rebuild
        _rebuild()

        font_path = '/System/Library/Fonts/Supplemental/AppleGothic.ttf'

        font_name = fm.FontProperties(fname=font_path).get_name()
        plt.rc('font', family=font_name)
        plt.rc('axes', unicode_minus=False)
        # 그래프에서 마이너스 폰트 깨지는 문제에 대한 대처
        mpl.rcParams['axes.unicode_minus'] = False
        #print('버전: ', mpl.__version__)
        #print('설치 위치: ', mpl.__file__)
        #print('설정 위치: ', mpl.get_configdir())
        #print('캐시 위치: ', mpl.get_cachedir())

        # size, family
        print('# 설정 되어있는 폰트 사이즈')
        print(plt.rcParams['font.size'])
        print('# 설정 되어있는 폰트 글꼴')
        print(plt.rcParams['font.family'])

        fig = plt.figure(figsize=(8, 8))
        pos = self.centrality_layout()

        """Conveniently summarize graph visually"""
        # config parameters
        edge_min_width= 3
        edge_max_width= 12
        label_font = 18
        node_font = 22
        node_alpha = 0.4
        edge_alpha = 0.55
        edge_cmap = plt.cm.Spectral

        # Create figure
        if fig is None:
            fig, ax = plt.subplots()
        else:
            ax = fig.add_subplot(111)
        fig.subplots_adjust(0,0,1)

        font = FT2Font(font_path)

        # Plot nodes with size according to count
        sizes = []
        degrees = []
        for n, d in self.G.nodes(data=True):
            sizes.append(d['count'])
            degrees.append(self.G.degree(n))

        sizes = self.rescale_arr(np.array(sizes, dtype=float), 100, 1000)

        # Compute layout and label edges according to weight
        pos = nx.spectral_layout(self.G) if pos is None else pos
        labels = {}
        width = []
        for n1, n2, d in self.G.edges(data=True):
            w = d['weight']
            labels[n1, n2] = w
            width.append(w)

        width = self.rescale_arr(np.array(width, dtype=float), edge_min_width,
                            edge_max_width)

        # Draw
        nx.draw_networkx_nodes(self.G, pos, node_size=sizes, node_color=degrees,
                               alpha=node_alpha)
        nx.draw_networkx_edges(self.G, pos, width=width, edge_color=width,
                               edge_cmap=edge_cmap, alpha=edge_alpha)
        #nx.draw_networkx_edge_labels(self.G, pos, edge_labels=labels,
                                     #font_size=label_font)
        nx.draw_networkx_labels(self.G, pos, font_size=node_font, font_family=font_name, font_weight='bold')

        if title is not None:
            ax.set_title(title, fontsize=label_font)
        ax.set_xticks([])
        ax.set_yticks([])

        # Mark centrality axes
        kw = dict(color='k', linestyle='-')
        cross = [ax.axhline(0, **kw), ax.axvline(self.rad0, **kw)]
        [l.set_zorder(0) for l in cross]
        
        plt.savefig(file)
        plt.show()

    def rescale_arr(self, arr, amin, amax):
        """Rescale an array to a new range.
        Return a new array whose range of values is (amin, amax).
        Parameters
        ----------
        arr : array-like
        amin : float
          new minimum value
        amax : float
          new maximum value
        Examples
        --------
        >>> a = np.arange(5)
        >>> rescale_arr(a,3,6)
        array([ 3.  ,  3.75,  4.5 ,  5.25,  6.  ])
        """

        # old bounds
        m = arr.min()
        M = arr.max()
        # scale/offset
        s = float(amax - amin) / (M - m)
        d = amin - s * m

        # Apply clip before returning to cut off possible overflows outside the
        # intended range due to roundoff error, so that we can absolutely guarantee
        # that on output, there are no values > amax or < amin.
        return np.clip(s * arr + d, amin, amax)

    def summarize_centrality(self, limit=10):
        centrality = nx.eigenvector_centrality_numpy(self.G)
        c = centrality.items()
        c = sorted(c, key=lambda x: x[1], reverse=True)
        print('\nGraph centrality')
        count=0
        for node, cent in c:
            if count>limit:
                break
            print ("%15s: %.3g" % (node, float(cent)))
            count+=1

    def sort_freqs(self, freqs):
        """Sort a word frequency histogram represented as a dictionary.
        Parameters
        ----------
        freqs : dict
          A dict with string keys and integer values.
        Return
        ------
        items : list
          A list of (count, word) pairs.
        """
        items = freqs.items()
        items.sort(key=lambda wc: wc[1])
        return items

    def plot_word_histogram(self, freqs, show=10, title=None):
        """Plot a histogram of word frequencies, limited to the top `show` ones.
        """
        sorted_f = self.sort_freqs(freqs) if isinstance(freqs, dict) else freqs

        # Don't show the tail
        if isinstance(show, int):
            # interpret as number of words to show in histogram
            show_f = sorted_f[-show:]
        else:
            # interpret as a fraction
            start = -int(round(show * len(freqs)))
            show_f = sorted_f[start:]

        # Now, extract words and counts, plot
        n_words = len(show_f)
        ind = np.arange(n_words)
        words = [i[0] for i in show_f]
        counts = [i[1] for i in show_f]

        fig = plt.figure()
        ax = fig.add_subplot(111)

        if n_words <= 20:
            # Only show bars and x labels for small histograms, they don't make
            # sense otherwise
            ax.bar(ind, counts)
            ax.set_xticks(ind)
            ax.set_xticklabels(words, rotation=45)
            fig.subplots_adjust(bottom=0.25)
        else:
            # For larger ones, do a step plot
            ax.step(ind, counts)

        # If it spans more than two decades, use a log scale
        if float(max(counts)) / min(counts) > 100:
            ax.set_yscale('log')

        if title:
            ax.set_title(title)
        return ax

In [59]:
cv = CountVectorizer()
cv_fit = cv.fit_transform(documents)
word_list = cv.get_feature_names();
count_list = cv_fit.toarray().sum(axis=0)
word_hist = dict(zip(word_list, count_list))

In [60]:
graph_builder = GraphMLCreator()
#graph_builder.createGraphMLWithThreshold(co_result, word_hist, vocab, file="/Users/angieryu2202/Desktop/IF_App_Study/yes_response_coword_network_1.graphml", threshold=1.0)
graph_builder.createGraphMLWithThreshold(co_result, word_hist, vocab, file="/Users/angieryu2202/Desktop/IF_App_Study/no_response_coword_network_1.graphml", threshold=1.0)

91


In [89]:
#graph_builder.plot_graph(file="/Users/angieryu2202/Desktop/IF_App_Study/if_app_review_coword_network.pdf")